In [1]:
import automol

rsmi, psmi = ("CCCO[O]", "[CH2]CCOO")
rsmis = automol.smiles.split(rsmi)
psmis = automol.smiles.split(psmi)

# Get the starting TS structure
rxn, *_ = automol.reac.from_smiles(rsmis, psmis, stereo=False, struc_typ="geom")
ts_gra = automol.reac.ts_graph(rxn)
ts_geo = automol.reac.ts_structure(rxn)

# Determine the breaking and forming bonds
brk_bkey, = automol.graph.ts.breaking_bond_keys(ts_gra)
frm_bkey, = automol.graph.ts.forming_bond_keys(ts_gra)

automol.graph.display(ts_gra, label=True, exp=True)
automol.geom.display(ts_geo, gra=ts_gra)
print(automol.geom.distance(ts_geo, *frm_bkey, angstrom=True))

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

1.824046106909827


In [2]:
# Sella TS optimization
from tblite.ase import TBLite
from sella import Sella

atms_obj = automol.geom.ase_atoms(ts_geo)
atms_obj.calc = TBLite(method="GFN1-xTB")

dyn = Sella(atms_obj, order=1, internal=True)
dyn.run()

geo_out = automol.geom.from_ase_atoms(atms_obj)
automol.geom.display(geo_out)
print(automol.geom.distance(geo_out, *frm_bkey, angstrom=True))

/home/avcopan/code/amech-dev/.pixi/envs/default/lib/python3.12/site-packages/ase/optimize/optimize.py:372: FutureWarning: force_consistent keyword is deprecated and will be ignored.  This will raise an error in future versions of ASE.
  warnings.warn(


------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -18.99057833730  -1.9167150E+01   5.3793479E-01
      2     -19.21567441147  -2.2509607E-01   3.3989348E-01
      3     -19.20704254664   8.6318648E-03   1.6074957E-01
      4     -19.22777691335  -2.0734367E-02   5.6557343E-02
      5     -19.23666016733  -8.8832540E-03   2.6427568E-02
      6     -19.23930951642  -2.6493491E-03   4.2408560E-03
      7     -19.23929384216   1.5674262E-05   2.6364911E-03
      8     -19.23929202588   1.8162756E-06   2.8004935E-03
      9     -19.23932836414  -3.6338255E-05   3.8978647E-04
     10     -19.23932889673  -5.3258707E-07   2.9796322E-04
     11     -19.23932929923  -4.0250320E-07   1.0188038E-04
     12     -19.23932935164  -5.2415654E-08   1.4591912E-05
------------------------------------------------------------

 total:                             

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

1.3320889578749693


In [3]:

# Sella equilibrium optimization
from tblite.ase import TBLite
from sella import Sella

atms_obj = automol.geom.ase_atoms(ts_geo)
atms_obj.calc = TBLite(method="GFN1-xTB")

dyn = Sella(atms_obj, order=0, internal=True)
dyn.run()

geo_out = automol.geom.from_ase_atoms(atms_obj)
automol.geom.display(geo_out)
print(automol.geom.distance(geo_out, *frm_bkey, angstrom=True))

/home/avcopan/code/amech-dev/.pixi/envs/default/lib/python3.12/site-packages/ase/optimize/optimize.py:372: FutureWarning: force_consistent keyword is deprecated and will be ignored.  This will raise an error in future versions of ASE.
  warnings.warn(


------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -18.99057833730  -1.9167150E+01   5.3793479E-01
      2     -19.21567441147  -2.2509607E-01   3.3989348E-01
      3     -19.20704254664   8.6318648E-03   1.6074957E-01
      4     -19.22777691335  -2.0734367E-02   5.6557343E-02
      5     -19.23666016733  -8.8832540E-03   2.6427568E-02
      6     -19.23930951642  -2.6493491E-03   4.2408560E-03
      7     -19.23929384216   1.5674262E-05   2.6364911E-03
      8     -19.23929202588   1.8162756E-06   2.8004935E-03
      9     -19.23932836414  -3.6338255E-05   3.8978647E-04
     10     -19.23932889673  -5.3258705E-07   2.9796322E-04
     11     -19.23932929923  -4.0250321E-07   1.0188038E-04
     12     -19.23932935164  -5.2415679E-08   1.4591912E-05
------------------------------------------------------------

 total:                             

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

2.2016817055961377


In [9]:
# Sella constrained optimization
from tblite.ase import TBLite
from sella import Sella, Internals, Constraints

atms_obj = automol.geom.ase_atoms(ts_geo)
atms_obj.calc = TBLite(method="GFN1-xTB")

cons_obj = Constraints(atms_obj)
cons_obj.fix_bond(tuple(frm_bkey))

ints_obj = Internals(atms_obj, cons=cons_obj)
ints_obj.find_all_bonds()
ints_obj.find_all_angles()
ints_obj.find_all_dihedrals()

dyn = Sella(atms_obj, order=0, internal=ints_obj)
dyn.run()

geo_out = automol.geom.from_ase_atoms(atms_obj)
automol.geom.display(geo_out)
print(automol.geom.distance(geo_out, *frm_bkey, angstrom=True))


/home/avcopan/code/amech-dev/.pixi/envs/default/lib/python3.12/site-packages/ase/optimize/optimize.py:372: FutureWarning: force_consistent keyword is deprecated and will be ignored.  This will raise an error in future versions of ASE.
  warnings.warn(


------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -18.99057833730  -1.9167150E+01   5.3793479E-01
      2     -19.21567441147  -2.2509607E-01   3.3989348E-01
      3     -19.20704254664   8.6318648E-03   1.6074957E-01
      4     -19.22777691335  -2.0734367E-02   5.6557343E-02
      5     -19.23666016733  -8.8832540E-03   2.6427568E-02
      6     -19.23930951642  -2.6493491E-03   4.2408560E-03
      7     -19.23929384216   1.5674262E-05   2.6364911E-03
      8     -19.23929202588   1.8162756E-06   2.8004935E-03
      9     -19.23932836414  -3.6338255E-05   3.8978647E-04
     10     -19.23932889673  -5.3258707E-07   2.9796322E-04
     11     -19.23932929923  -4.0250321E-07   1.0188038E-04
     12     -19.23932935164  -5.2415650E-08   1.4591912E-05
------------------------------------------------------------

 total:                             

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

1.8240440499255544
